In [ ]:
# Common imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

# Networking/packet analysis
import pyshark
import scapy.all as scapy
import networkx as nx

# Helper: inline plots
%matplotlib inline


# WireScope — Transmission Line Reflections

**Goal:** Compute reflection coefficient Γ and visualize standing waves.


In [ ]:
# TODO: Implement Γ = (Z_load - Z0)/(Z_load + Z0).
# TODO: Plot step response for ZL=25Ω, 75Ω, open.
# HINT: Use np.convolve with step function to simulate reflections.
